# CIV1538: My First Logit Model
This tutorial will give you through the process of creating and estimating a discrete choice model using the Python package Biogeme. Thank you to Professor Michel Bierlaire of EPFL for making his data, code, and examples available for public use.

Your first task is to install Python on your computer for use of the Biogeme package. This can be downloaded as an executable file from the Python website for Windows and Mac machines (Linux comes preloaded with Python). You will need to install Python version 3.7+ for the easiest installation of Biogeme.

Once you have Python installed on your computer, make sure it is working correctly by going to the command prompt/terminal and typing: ```python```. This should give you an output like: ![title](./img/python_command.png)
You also need to have install pip (a Python package manager), which can be selected as an option during the install process. Biogeme can then be installed using the command: ```pip install Biogeme```. This will install the Biogeme package, as well as all its dependency Python packages. Once you have everything correctly installed on your computer, you can continue to the writing of the estimation script. I breakout the components of the script below. 
# 01logit.py Basic Biogeme Script
The first step is to import the necessary packages: Pandas, Biogeme, and headers.

In [ ]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from headers import *

The next step is to import the estimation data. Biogeme was recently updated to use the excellent package Pandas for data preparation. I import the swissmetro data as a Pandas table, then add it as a database within the internal Biogeme code structure. We can view the first few rows of the data using the command: ```database.data.head()```

In [20]:
pandas = pd.read_table("swissmetro.dat")
database = db.Database("swissmetro",pandas)
database.data.head()

GROUP  SURVEY  SP  ID  PURPOSE  FIRST  TICKET  WHO  LUGGAGE  AGE   ...    \
0      2       0   1   1        1      0       1    1        0    3   ...     
1      2       0   1   1        1      0       1    1        0    3   ...     
2      2       0   1   1        1      0       1    1        0    3   ...     
3      2       0   1   1        1      0       1    1        0    3   ...     
4      2       0   1   1        1      0       1    1        0    3   ...     

   TRAIN_TT  TRAIN_CO  TRAIN_HE  SM_TT  SM_CO  SM_HE  SM_SEATS  CAR_TT  \
0       112        48       120     63     52     20         0     117   
1       103        48        30     60     49     10         0     117   
2       130        48        60     67     58     30         0     117   
3       103        40        30     63     52     20         0      72   
4       130        36        60     63     42     20         0      90   

   CAR_CO  CHOICE  
0      65       2  
1      84       2  
2      52       2  
3      52       2  
4      84       2  

[5 rows x 28 columns]

Removing some observations can be done directly using pandas.

In [ ]:
print("Size of df before removals: ", database.data.shape)
remove = (((database.data.PURPOSE != 1) & (database.data.PURPOSE != 3)) | (database.data.CHOICE == 0))
database.data.drop(database.data[remove].index,inplace=True)
print("Size of df after removals: ", database.data.shape)

Size of df before removals:  (10728, 28)
Size of df after removals:  (6768, 28)


We can then define the names and starting values for the model parameters. There are 5 parameters defined for the base Swiss Metro model. Biogeme uses a constrained optimization to reduce the search space. The Beta function is called as follows:
```python
Beta(parameter name AS str, initial value AS int, lower bound AS int, upper bound AS int, fixed AS int)
```
If there are no bounds on the parameter search space, the use may enter: ```None```. A value of ```0``` for the ```fixed``` parameter will allow Biogeme to estimate the parameter. In this case, we have only fixed the ASC for the swiss metro mode, to maintain model indentification.

In [4]:
ASC_CAR = Beta('ASC_CAR',0,None,None,0)
ASC_TRAIN = Beta('ASC_TRAIN',0,None,None,0)
ASC_SM = Beta('ASC_SM',0,None,None,1)
B_TIME = Beta('B_TIME',0,None,None,0)
B_COST = Beta('B_COST',0,None,None,0)

We can then define the corresponding variables used in the model. Biogeme has an internal set of functions that are used in the definition of variables. The user can scale a variable value or modify its value using boolean logic.

In [5]:
SM_COST =  SM_CO   * (  GA   ==  0  ) 
TRAIN_COST =  TRAIN_CO   * (  GA   ==  0  )

CAR_AV_SP =  DefineVariable('CAR_AV_SP',CAR_AV  * (  SP   !=  0  ),database)
TRAIN_AV_SP =  DefineVariable('TRAIN_AV_SP',TRAIN_AV  * (  SP   !=  0  ),database)

TRAIN_TT_SCALED = DefineVariable('TRAIN_TT_SCALED',\
                                 TRAIN_TT / 100.0,database)
TRAIN_COST_SCALED = DefineVariable('TRAIN_COST_SCALED',\
                                   TRAIN_COST / 100,database)
SM_TT_SCALED = DefineVariable('SM_TT_SCALED', SM_TT / 100.0,database)
SM_COST_SCALED = DefineVariable('SM_COST_SCALED', SM_COST / 100,database)
CAR_TT_SCALED = DefineVariable('CAR_TT_SCALED', CAR_TT / 100,database)
CAR_CO_SCALED = DefineVariable('CAR_CO_SCALED', CAR_CO / 100,database)

The next step is to define the utility function for each alternative mode.

In [6]:
V1 = ASC_TRAIN + \
     B_TIME * TRAIN_TT_SCALED + \
     B_COST * TRAIN_COST_SCALED
V2 = ASC_SM + \
     B_TIME * SM_TT_SCALED + \
     B_COST * SM_COST_SCALED
V3 = ASC_CAR + \
     B_TIME * CAR_TT_SCALED + \
     B_COST * CAR_CO_SCALED

Biogeme has a variety of built-in discrete choice models, which the user can leverage. However, it also allows the user to define their own likelihood function if it is not available in the built-in model library. In this case, we use the built-in ```bioLogLogit``` function to estimate the model.

In [7]:
# Associate utility functions with the numbering of alternatives
V = {1: V1,
     2: V2,
     3: V3}

# Associate the availability conditions with the alternatives

av = {1: TRAIN_AV_SP,
      2: SM_AV,
      3: CAR_AV_SP}

logprob = bioLogLogit(V,av,CHOICE)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "01logit"
results = biogeme.estimate()

The results of the model estimation can then be printed to the terminal.

In [8]:
# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_CAR=	-0.155
ASC_TRAIN=	-0.701
B_COST=	-1.08
B_TIME=	-1.28
              Value   Std err     t-test   p-value  Rob. Std err  Rob. t-test  \
ASC_CAR   -0.154643  0.043236  -3.576747  0.000348      0.058164    -2.658749   
ASC_TRAIN -0.701195  0.054874 -12.778246  0.000000      0.082563    -8.492878   
B_COST    -1.083788  0.051830 -20.910332  0.000000      0.068225   -15.885444   
B_TIME    -1.277878  0.056884 -22.464739  0.000000      0.104256   -12.257139   

           Rob. p-value  
ASC_CAR        0.007843  
ASC_TRAIN      0.000000  
B_COST         0.000000  
B_TIME         0.000000  
